In [1]:
import sys
import os
import numpy as np

import timeit
sys.path.append("/Users/zhouji/Documents/github/gempy")


import matplotlib.pyplot as plt
import gempy as gp
from gempy.core.tensor.tensorflow_graph_test import TFGraph
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
from gempy import create_data, map_series_to_surfaces
from gempy.assets.geophysics import GravityPreprocessing
tfd = tfp.distributions

def Plot_2D_scaler_field(grid, scaler_field):
    G = grid[np.where(grid[:, 1] == [grid[-1][1]])[0]]
    S = scaler_field.numpy()[np.where(grid[:, 1] == [grid[0][1]])[0]]
    XX = G[:, 0].reshape([50, 50])
    ZZ = G[:, 2].reshape([50, 50])
    S = S.reshape([50, 50])
    plt.contour(XX, ZZ, S)

path = '/Users/zhouji/Documents/github/gempy/notebooks/'
geo_data = create_data([0, 1000, 0, 1000, 0, 1000], resolution=[50, 50, 50],
                       path_o="/Users/zhouji/Google Drive/Gempy/check.csv",
                       path_i=path + "/data/input_data/jan_models/model2_surface_points.csv")
map_series_to_surfaces(geo_data, {"Strat_Series": (
    'rock2', 'rock1'), "Basement_Series": ('basement')})

geo_data.add_surface_values([2.61, 3.1, 2.92])

# Gravity test
# ---------
grav_res = 15
X = np.linspace(0, 1000, grav_res)
Y = np.linspace(0, 1000, grav_res)
Z = 300
xyz = np.meshgrid(X, Y, Z)
xy_ravel = np.vstack(list(map(np.ravel, xyz))).T


geo_data.set_centered_grid(xy_ravel, resolution=[10, 10, 15], radius=5000)
interpolator = geo_data.interpolator
dtype = interpolator.additional_data.options.df.loc['values', 'dtype']
dtype = 'float64'
dips_position, dip_angles, azimuth, polarity, surface_points_coord, fault_drift, grid, values_properties = interpolator.get_python_input_block()[
    0:-3]
dip_angles = tf.cast(dip_angles,dtype)
grid = tf.cast(grid,dtype)
dips_position = tf.cast(dips_position,dtype)
azimuth = tf.cast(azimuth,dtype)
polarity = tf.cast(polarity,dtype)
fault_drift = tf.cast(fault_drift,dtype)
values_properties = tf.cast(values_properties,dtype)


centers = geo_data.rescaling.df.loc['values', 'centers'].astype(dtype)


g = GravityPreprocessing(geo_data.grid.centered_grid)
tz = g.set_tz_kernel()

tz = tf.cast(tz,dtype)

len_rest_form = interpolator.additional_data.structure_data.df.loc[
    'values', 'len surfaces surface_points'] - 1
Range = interpolator.additional_data.kriging_data.df.loc['values', 'range']
C_o = interpolator.additional_data.kriging_data.df.loc['values', '$C_o$']
rescale_factor = interpolator.additional_data.rescaling_data.df.loc[
    'values', 'rescaling factor']

# rf = rescale_factor
nugget_effect_grad = np.cast[dtype](
    np.tile(interpolator.orientations.df['smooth'], 3))
nugget_effect_scalar = np.cast[dtype](
    interpolator.surface_points.df['smooth'])


TFG = TFGraph(dips_position, dip_angles, azimuth,
              polarity, fault_drift,
              grid, values_properties, len_rest_form, Range,
              C_o, nugget_effect_scalar, nugget_effect_grad,
              rescale_factor)


surface_points_coord = tf.convert_to_tensor(surface_points_coord,TFG.dtype)

Z_x= TFG.scalar_field(surface_points_coord)
scalar_field_at_surface_points = TFG.get_scalar_field_at_surface_points(Z_x)
formations_block = TFG.export_formation_block(
    Z_x, scalar_field_at_surface_points, values_properties)

lg_0 = interpolator.grid.get_grid_args('centered')[0]
lg_1 = interpolator.grid.get_grid_args('centered')[1]
dips_position = tf.convert_to_tensor(dips_position,dtype=dtype)
dip_angles = tf.convert_to_tensor(dip_angles,dtype=dtype)
azimuth = tf.convert_to_tensor(azimuth,dtype=dtype)
polarity = tf.convert_to_tensor(polarity,dtype=dtype)

fault_drift = tf.convert_to_tensor(fault_drift,dtype=dtype)
grid = tf.convert_to_tensor(grid,dtype=dtype)
values_properties = tf.convert_to_tensor(values_properties,dtype=dtype)
len_rest_form = tf.convert_to_tensor(len_rest_form,dtype=dtype)
Range = tf.convert_to_tensor(Range, dtype)
C_o = tf.convert_to_tensor(C_o,dtype=dtype)
nugget_effect_grad = tf.convert_to_tensor(nugget_effect_grad,dtype=dtype)
nugget_effect_scalar = tf.convert_to_tensor(nugget_effect_scalar,dtype=dtype)
rescale_factor = tf.convert_to_tensor(rescale_factor, dtype)

densities = formations_block[1][lg_0:lg_1]

## Plot the ground truth
grav = TFG.compute_forward_gravity(tz, lg_0, lg_1, densities)
Data = grav

Active grids: ['regular']


AssertionError: One or more columns do not match with the expected values, which are: 
- the locations of the measurement points 'X','Y' and 'Z' 
- EITHER 'azimuth' (trend direction indicated by an angle between 0 and 360 with North at 0 AND 'dip' (inclination angle, measured from horizontal plane downwards, between 0 and 90 degrees) 
- OR the pole vectors of the orientation in a cartesian system 'G_x','G_y' and 'G_z' 
- the 'polarity' of the orientation, can be normal (1) or reversed (-1) 
- the name of the surface: 'formation'
Your headers are ['X', 'Y', 'Z', 'formation']